# h1_ENB_OCC.ipynb
Find all interactions in the original dataset for one issue or a range of issues.

In [10]:
import import_ipynb
import pandas as pd
import csv

In [11]:
def open_dict_interaction():
    dict_entities = {}
    with open("Files/dict_inter.csv") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            dict_entities[row[0]] = row[1].split(', ')
    return dict_entities      

In [12]:
def create_dict():
    """Create dictionary that can map entities with an id but also map entities from the original to the generated dataset. """
    DICTIONARY = open_dict_interaction()
    n = 0
    DICTIONARY_NUM = {}
    for k in DICTIONARY:
        DICTIONARY_NUM[k] = n 
        n +=1
    NAMES = []
    for x in list(DICTIONARY.values()):
        if(len(x) == 1):
            NAMES.append(x[0])
        else:
            NAMES += [k for k in x]

    return DICTIONARY, DICTIONARY_NUM, NAMES

In [13]:
DICTIONARY, DICTIONARY_NUM, NAMES = create_dict()

In [14]:
def create_df(cooperations,issue_number):
    """ Function that combines all the information of the interaction to a dataframe. """
    ca = [x[0] for x in cooperations]
    cb = [x[1] for x in cooperations]
    id_ca = [x[17] for x in cooperations]
    id_cb = [x[18] for x in cooperations]
    behalf = [int(float(x[5])) for x in cooperations]
    support = [int(float(x[6]))for x in cooperations]
    agreement =[int(float(x[8]))for x in cooperations]
    opposition =[int(float(x[10]))for x in cooperations]
    criticism =[int(float(x[11])) for x in cooperations]
    cooperation =[int(float(x[12])) for x in cooperations]
    sentences = [x[-3] for x in cooperations]
    dict_issue = {'type':'original','issue': issue_number,'id_ca':id_ca,'id_cb':id_cb,'Country A':ca, 'Country B': cb, 'behalf':behalf,'support':support,'agreement':agreement,'opposition':opposition,'criticism':criticism,'cooperation':cooperation, 'sentences':sentences}
    df = pd.DataFrame(dict_issue)
    return df

In [15]:
def extract_from_csv_list_issues(csv_file):
    """ Extract from "csv_file" all the html link to be able to extract all the <p> tags. """
    f = open(csv_file)
    csv_f = csv.reader(f)
    list_pt = []
    return list(csv_f)[1:]

In [16]:
l = extract_from_csv_list_issues("Text/ENB_relationships.txt")

In [17]:
def extract_date(issue_number):
    """ Function that find the specific date of one issue. """
    list_meetings = extract_from_csv_list_issues('Text/list_meetings.csv')
    for i in range(len(list_meetings)):

        if(list_meetings[i][4] == str(issue_number)):

            date = list_meetings[i][5].replace('-','')

            return date

In [18]:
def extract_relationships_count(issue_number):
    """ Function that find all the interaction of one issue. """
    lines = []
    sentences = []
    data = []
    entities = []
    date = extract_date(issue_number)
    with open("Text/ENB_relationships.txt") as infile:
        k=0
        for line in infile:
            x = line.replace('"','').split('\t')[:26]
            entities.append(x[4])
            entities.append(x[3])
            if(k > 0 and date in x and k!=0):
                lines.append(line)
            k += 1
        se_old = ''
    for l in lines:
        list_line = l.replace('"','').split('\t')[:26]
            
        
        informations = list_line[3:19]
        informations = [f.upper() for f in informations]
        se_new = list_line[25]

        if(se_new != ''):

            se_old = se_new
            sentences.append(se_old)
            informations.append(se_old)
        
        else:

            sentences.append(se_old)
            informations.append(se_old)
        id_cb = 8888
        id_ca = 8888
        #print('Countries :',list_line[3],list_line[4])
        if(list_line[3] in DICTIONARY_NUM.keys()):
            id_ca = DICTIONARY_NUM[list_line[3]]

        if(list_line[4] in DICTIONARY_NUM.keys()):
            id_cb = DICTIONARY_NUM[list_line[4]]

        if(id_ca == 8888):
            print(list_line[3])
        informations.append(id_ca)
        informations.append(id_cb)
        data.append(informations)

    return create_df(data,issue_number), sentences

In [20]:
df, sent = extract_relationships_count(50)

In [21]:
df

,type,issue,id_ca,id_cb,Country A,Country B,behalf,support,agreement,opposition,criticism,cooperation,sentences
0,original,50,29,79,BRAZIL,G77,0,0,0,1,0,0,He (G77/China) indicated that an agreement ent...
1,original,50,180,75,SLOVENIA,EU,0,1,0,0,0,1,SLOVENIA signaled its preparation to eventuall...


In [22]:
sent

['He (G77/China) indicated that an agreement entailed advancing commitments for Annex I Parties, while avoiding new ones for non-Annex I Parties. He (Brazil) called for the development of quantitative targets for non-Annex I countries as they rea',
 'SLOVENIA signaled its preparation to eventually join Annex I Parties in complying with legally binding commitments and supported an EU proposal for GHG reductions.']